<a href="https://colab.research.google.com/github/xiborxu/IST462xibo/blob/main/Chicago%20App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Crash_Reporting_-_Drivers_Data-2.csv')

In [ ]:
value_counts = df['Agency Name'].value_counts() # Count the values of each agency name
plt.figure(figsize=(8, 6))
value_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Agency Name')
plt.ylabel('Report Value Counts')
plt.title('Report Cases by Agency Name')
plt.show()

In [ ]:
columns_drop = ['Road Name','Cross-Street Name','Report Number', 'Local Case Number', 'Off-Road Description', 'Municipality', 'Related Non-Motorist','Drivers License State', 'Non-Motorist Substance Abuse', 'Person ID', 'Circumstance','Vehicle ID']
df = df.drop(columns_drop, axis=1)

In [ ]:
columns_to_fill_UNKNOWN = ['Driver Substance Abuse', 'Traffic Control','Weather','Vehicle Damage Extent','Equipment Problems','Surface Condition','Collision Type','Light','Vehicle First Impact Location','Vehicle Second Impact Location','Vehicle Movement','Vehicle Make','Vehicle Model','Vehicle Body Type']
df[columns_to_fill_UNKNOWN] = df[columns_to_fill_UNKNOWN].fillna('UNKNOWN')
columns_to_fill_Unknown = ['Route Type','Cross-Street Type','Vehicle Continuing Dir','Vehicle Going Dir']
df[columns_to_fill_Unknown] = df[columns_to_fill_Unknown].fillna('Unknown')
df['Vehicle Make'] = df['Vehicle Make'].replace('TOYT', 'TOYOTA')
df['Vehicle Make'] = df['Vehicle Make'].replace('HOND', 'HONDA')
df['Vehicle Make'] = df['Vehicle Make'].replace('CHEV', 'CHEVROLET')
df['Vehicle Make'] = df['Vehicle Make'].replace('CHEVY', 'CHEVROLET')

In [ ]:
def generate_bar_graph(column):
    plt.figure(figsize=(8, 6))
    df[column].value_counts().sort_index().plot(kind='bar', color='blue')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.show()

In [ ]:
generate_bar_graph('Weather')

In [ ]:
generate_bar_graph('Speed Limit')

In [ ]:
def categorize_year(year):
    if year >= 2020:
        return '2020-2023'
    elif year >= 2017:
        return '2017-2020'
    elif year >= 2012:
        return '2012-2017'
    elif year >= 2008:
        return '2008-2012'
    elif year >= 2000:
        return '2000-2008'
    else:
        return '2000 and before'
df['Year Range'] = df['Vehicle Year'].apply(categorize_year)
generate_bar_graph('Year Range')

In [ ]:
# draw the graph of damage count by damaged level in different weather conditions
grouped = df.groupby(['Vehicle Damage Extent', 'Weather']).size().unstack(fill_value=0)
# groupby the Vehicle Damage Extent and weather and count the size
grouped.plot(kind='bar', stacked=True, figsize=(8, 6),colormap = 'tab10')
plt.xlabel('Weather')
plt.ylabel('Count')
plt.title('Damage Extent by Weather')
plt.legend(title='Damage Extent',bbox_to_anchor=(1.45, 1), loc='upper right')

In [ ]:
top_5_brands = df['Vehicle Make'].value_counts().head(5).index.tolist()

# Filter the data for the top 5 brands
filtered_df = df[df['Vehicle Make'].isin(top_5_brands)]
grouped = filtered_df.groupby(['Vehicle Make', 'Vehicle Damage Extent']).size().unstack(fill_value=0)
percentage_grouped = grouped.div(grouped.sum(axis=1), axis=0) * 100
# Calculate the percentage for each damage extent level within each brand
# draw the graph
ax = percentage_grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
for rect in ax.patches:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, rect.get_y() + height / 2,
            f'{height:.1f}%', ha='center', va='center')
plt.xlabel('Brand')
plt.ylabel('Percentage (%)')
plt.title('Damage Extent Distribution for Top 5 Brands')
plt.legend(title='Damage Extent', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)

In [ ]:
#df.dtypes
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns: # convert all object columns to integer
    df[column] = le.fit_transform(df[column])

In [ ]:
corr = df.corr()
corr_with_damage = corr

# Create a styled correlation map for that column
corr_with_damage.style.background_gradient(cmap='coolwarm')